In [7]:
import cv2 
import mediapipe as mp  
from PIL import ImageFont, ImageDraw, Image
import numpy as np      



def draw_ball_location(image, locations, color):
    for i in range(len(locations)-1):

        if locations[0] is None or locations[i+1] is None:
            continue

        cv2.line(image, tuple(locations[i]), tuple(locations[i+1]), color, 3)

    return image

list_ball_location = []
history_ball_locations = []

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
mp_drawing_styles = mp.solutions.drawing_styles


cap = cv2.VideoCapture(0)

text = ""
color = (0, 0, 0)  # default color : black (BGR)
line_color = color

with mp_hands.Hands(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:

 
  while cap.isOpened():
    success, image = cap.read()

    if not success:
      print("Ignoring empty camera frame.")
      continue

    # Flip the image horizontally for a later selfie-view display, and convert
    # the BGR image to RGB.
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    results = hands.process(image)


    # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    image_height, image_width, _ = image.shape


    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
       
        # 엄지를 제외한 나머지 4개 손가락의 마디 위치 관계를 확인하여 플래그 변수를 설정합니다. 손가락을 일자로 편 상태인지 확인합니다.
        thumb_finger_state = 0  # 변수를 0으로 초기화
        # 이미지의 높이와 좌표를 곱하는 이유 : 상대적인 위치를 비교하기 위함
        # --> 손 모양 인식에서는 이미지의 크기가 다양할 수 있음 --> 절대적인 좌표 값만으로는 손 모양 위치와 상대적인 관계를 파악하기 어렵다
        # 이미지의 높이를 곱하는 것 : 상대적인 비율을 활용하여 손 관절의 위치를 비교하는 것을 의미
        if hand_landmarks.landmark[1].y * image_height > hand_landmarks.landmark[2].y * image_height:
          if hand_landmarks.landmark[2].y * image_height > hand_landmarks.landmark[3].y * image_height:
            if hand_landmarks.landmark[3].y * image_height > hand_landmarks.landmark[4].y * image_height:
              thumb_finger_state = 1

        index_finger_state = 0
        if hand_landmarks.landmark[5].y * image_height > hand_landmarks.landmark[6].y * image_height:
          if hand_landmarks.landmark[6].y * image_height > hand_landmarks.landmark[7].y * image_height:
            if hand_landmarks.landmark[7].y * image_height > hand_landmarks.landmark[8].y * image_height:
              index_finger_state = 1

        middle_finger_state = 0
        if hand_landmarks.landmark[9].y * image_height > hand_landmarks.landmark[10].y * image_height:
          if hand_landmarks.landmark[10].y * image_height > hand_landmarks.landmark[11].y * image_height:
            if hand_landmarks.landmark[11].y * image_height > hand_landmarks.landmark[12].y * image_height:
              middle_finger_state = 1

        ring_finger_state = 0
        if hand_landmarks.landmark[13].y * image_height > hand_landmarks.landmark[14].y * image_height:
          if hand_landmarks.landmark[14].y * image_height > hand_landmarks.landmark[15].y * image_height:
            if hand_landmarks.landmark[15].y * image_height > hand_landmarks.landmark[16].y * image_height:
              ring_finger_state = 1

        pinky_finger_state = 0
        if hand_landmarks.landmark[17].y * image_height > hand_landmarks.landmark[18].y * image_height:
          if hand_landmarks.landmark[18].y * image_height > hand_landmarks.landmark[19].y * image_height:
            if hand_landmarks.landmark[19].y * image_height > hand_landmarks.landmark[20].y * image_height:
              pinky_finger_state = 1

        use_pen = False
        if hand_landmarks.landmark[4].y * image_height > hand_landmarks.landmark[7].y * image_height:
          if hand_landmarks.landmark[12].y * image_height > hand_landmarks.landmark[7].y * image_height:
            if hand_landmarks.landmark[16].y * image_height > hand_landmarks.landmark[7].y * image_height:
              if hand_landmarks.landmark[20].y * image_height > hand_landmarks.landmark[7].y * image_height:
                if index_finger_state:
                  use_pen = True
       
        use_eraser = False
        if thumb_finger_state == 1 and index_finger_state == 1 and middle_finger_state == 1 and ring_finger_state == 1 and pinky_finger_state == 1:
          use_eraser = True

        use_move = False
        if   index_finger_state == 0 and middle_finger_state == 0 and ring_finger_state == 0 and pinky_finger_state == 0:
          use_move = True
          
        c_color = False
        if hand_landmarks.landmark[4].y * image_height > hand_landmarks.landmark[7].y * image_height:
          if hand_landmarks.landmark[16].y * image_height > hand_landmarks.landmark[7].y * image_height:
            if hand_landmarks.landmark[20].y * image_height > hand_landmarks.landmark[7].y * image_height:
              c_color = True
              
        r_color = False
        if hand_landmarks.landmark[4].y * image_height > hand_landmarks.landmark[19].y * image_height:
          if hand_landmarks.landmark[15].y * image_height > hand_landmarks.landmark[19].y * image_height:
            if ring_finger_state == 0:
              r_color = True
                
        b_color = False
        if hand_landmarks.landmark[4].y * image_height > hand_landmarks.landmark[15].y * image_height:
          if hand_landmarks.landmark[4].y * image_height > hand_landmarks.landmark[19].y * image_height:
            b_color = True
                
        g_color = False
        if hand_landmarks.landmark[4].y * image_height > hand_landmarks.landmark[15].y * image_height:
          if hand_landmarks.landmark[19].y * image_height:
            g_color = True

        image = Image.fromarray(image)
        draw = ImageDraw.Draw(image)

       
        text = ""
        if use_pen == True:
          text = "pen"

        elif use_eraser == True:
          text = "eraser"
       
        elif use_move == True:
          text = "move"
        
        elif c_color == True:
          text = 'light blue'
          
        elif r_color == True:
          text = 'red'
          
        elif b_color == True:
          text = 'black'
          
        elif g_color == True:
          text = 'green'
       

        if text == 'pen':
            x = int(hand_landmarks.landmark[8].x * image_width)
            y = int(hand_landmarks.landmark[8].y * image_height)

            list_ball_location.append((x,y))
           
       
        elif text == 'move':
          history_ball_locations.append(list_ball_location.copy())
          list_ball_location.clear()

        elif text == 'eraser':
           
            history_ball_locations.clear()
            list_ball_location.clear()
            
        elif text == 'light blue':
          color = (255, 255, 0)
          
        elif text == 'red':
          color = (0, 0, 255)
          
        elif text == 'black':
          color = (0, 0, 0)
          
        elif text == 'green':
          color = (0, 255, 0)
          

        font = ImageFont.truetype("fonts/gulim.ttc", 50)
        w, h = font.getsize(text)

        x = 50
        y = 50

        draw.rectangle((x, y, x + w, y + h), fill='black')
        draw.text((x, y),  text, font=font, fill=(255, 255, 255))
        image = np.array(image)

        # 손가락 뼈대를 그려줍니다.
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())

    if len(list_ball_location) > 1:
      image = draw_ball_location(image, list_ball_location[:-1], color)
      cv2.line(image, tuple(list_ball_location[-2]), tuple(list_ball_location[-1]), line_color, 3)
      
    for ball_locations in history_ball_locations:
      image = draw_ball_location(image, ball_locations, color)

    cv2.imshow('MediaPipe Hands', image)

    key = cv2.waitKey(5)
    if key == ord('q'):
      break

cap.release()
cv2.destroyAllWindows()  # 열린 모든 창을 닫음 

C:\Users\USER\AppData\Local\Temp\ipykernel_25712\3035015834.py:192: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  w, h = font.getsize(text)


In [12]:
hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_CMC]
# hand_landmarks.landmark[1]

x: 0.8184624314308167
y: 0.92503422498703
z: -0.025727257132530212

In [13]:
hand_landmarks.landmark[1].y

0.92503422498703

In [8]:
image_height

480

In [9]:
hand_landmarks.landmark[1].y * image_height

444.0164279937744

In [11]:
hand_landmarks.landmark[2].y

0.9849026203155518

In [14]:
hand_landmarks.landmark[2].y * image_height

472.75325775146484

In [2]:
import cv2 
import mediapipe as mp  
from PIL import ImageFont, ImageDraw, Image
import numpy as np      



def draw_ball_location(image, locations):
    for i in range(len(locations)-1):

        if locations[0] is None or locations[1] is None:
            continue

        cv2.line(image, tuple(locations[i]), tuple(locations[i+1]), (0, 255, 255), 3)

    return image

list_ball_location = []
history_ball_locations = []

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
mp_drawing_styles = mp.solutions.drawing_styles


cap = cv2.VideoCapture(0)

text = ""

with mp_hands.Hands(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:

 
  while cap.isOpened():
    success, image = cap.read()

    if not success:
      print("Ignoring empty camera frame.")
      continue

    # Flip the image horizontally for a later selfie-view display, and convert
    # the BGR image to RGB.
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    results = hands.process(image)


    # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    image_height, image_width, _ = image.shape


    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
       
        # 엄지를 제외한 나머지 4개 손가락의 마디 위치 관계를 확인하여 플래그 변수를 설정합니다. 손가락을 일자로 편 상태인지 확인합니다.
        thumb_finger_state = 0  # 변수를 0으로 초기화
        # 이미지의 높이와 좌표를 곱하는 이유 : 상대적인 위치를 비교하기 위함
        # --> 손 모양 인식에서는 이미지의 크기가 다양할 수 있음 --> 절대적인 좌표 값만으로는 손 모양 위치와 상대적인 관계를 파악하기 어렵다
        # 이미지의 높이를 곱하는 것 : 상대적인 비율을 활용하여 손 관절의 위치를 비교하는 것을 의미
        if hand_landmarks.landmark[1].y * image_height > hand_landmarks.landmark[2].y * image_height:
          if hand_landmarks.landmark[2].y * image_height > hand_landmarks.landmark[3].y * image_height:
            if hand_landmarks.landmark[3].y * image_height > hand_landmarks.landmark[4].y * image_height:
              thumb_finger_state = 1

        index_finger_state = 0
        if hand_landmarks.landmark[5].y * image_height > hand_landmarks.landmark[6].y * image_height:
          if hand_landmarks.landmark[6].y * image_height > hand_landmarks.landmark[7].y * image_height:
            if hand_landmarks.landmark[7].y * image_height > hand_landmarks.landmark[8].y * image_height:
              index_finger_state = 1

        middle_finger_state = 0
        if hand_landmarks.landmark[9].y * image_height > hand_landmarks.landmark[10].y * image_height:
          if hand_landmarks.landmark[10].y * image_height > hand_landmarks.landmark[11].y * image_height:
            if hand_landmarks.landmark[11].y * image_height > hand_landmarks.landmark[12].y * image_height:
              middle_finger_state = 1

        ring_finger_state = 0
        if hand_landmarks.landmark[13].y * image_height > hand_landmarks.landmark[14].y * image_height:
          if hand_landmarks.landmark[14].y * image_height > hand_landmarks.landmark[15].y * image_height:
            if hand_landmarks.landmark[15].y * image_height > hand_landmarks.landmark[16].y * image_height:
              ring_finger_state = 1

        pinky_finger_state = 0
        if hand_landmarks.landmark[17].y * image_height > hand_landmarks.landmark[18].y * image_height:
          if hand_landmarks.landmark[18].y * image_height > hand_landmarks.landmark[19].y * image_height:
            if hand_landmarks.landmark[19].y * image_height > hand_landmarks.landmark[20].y * image_height:
              pinky_finger_state = 1

        use_pen = False
        if hand_landmarks.landmark[4].y * image_height > hand_landmarks.landmark[7].y * image_height:
          if hand_landmarks.landmark[12].y * image_height > hand_landmarks.landmark[7].y * image_height:
            if hand_landmarks.landmark[16].y * image_height > hand_landmarks.landmark[7].y * image_height:
              if hand_landmarks.landmark[20].y * image_height > hand_landmarks.landmark[7].y * image_height:
                if index_finger_state:
                  use_pen = True
       
        use_eraser = False
        if thumb_finger_state == 1 and index_finger_state == 1 and middle_finger_state == 1 and ring_finger_state == 1 and pinky_finger_state == 1:
          use_eraser = True

        use_move = False
        if   index_finger_state == 0 and middle_finger_state == 0 and ring_finger_state == 0 and pinky_finger_state == 0:
          use_move = True


        image = Image.fromarray(image)
        draw = ImageDraw.Draw(image)

       
        text = ""
        if use_pen == True:
          text = "pen"

        elif use_eraser == True:
          text = "eraser"
       
        elif use_move == True:
          text = "move"
       

        if text == 'pen':
            x = int(hand_landmarks.landmark[8].x * image_width)
            y = int(hand_landmarks.landmark[8].y * image_height)

            list_ball_location.append((x,y))
           
       
        elif text == 'move':
          history_ball_locations.append(list_ball_location.copy())
          list_ball_location.clear()

        elif text == 'eraser':
           
            history_ball_locations.clear()
            list_ball_location.clear()

        font = ImageFont.truetype("fonts/gulim.ttc", 80)
        w, h = font.getsize(text)

        x = 50
        y = 50

        draw.rectangle((x, y, x + w, y + h), fill='black')
        draw.text((x, y),  text, font=font, fill=(255, 255, 255))
        image = np.array(image)

        # 손가락 뼈대를 그려줍니다.
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())

    image = draw_ball_location(image, list_ball_location)

    for ball_locations in history_ball_locations:
      image = draw_ball_location(image, ball_locations)

    cv2.imshow('MediaPipe Hands', image)

    key = cv2.waitKey(5)
    if key == ord('q'):
      break

cap.release()
cv2.destroyAllWindows()  # 열린 모든 창을 닫음 

C:\Users\USER\AppData\Local\Temp\ipykernel_436\3793558308.py:144: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  w, h = font.getsize(text)
